# wordnetを利用したオーバーサンプリング

In [1]:
%cd data/

/mnt/data1/home/yoshida/my_research/data


In [2]:
#!wget -O wnjpn.db.gz http://compling.hss.ntu.edu.sg/wnja/data/1.1/wnjpn.db.gz
#!gzip -d wnjpn.db.gz

In [3]:
import wordnet_jp as jp
import pprint 
import sqlite3
import subprocess
import MeCab
import pandas as pd
from sklearn.model_selection import train_test_split
import random

In [4]:
conn = sqlite3.connect("./wnjpn.db")

In [5]:
word = '犬'
synonym = jp.getSynonym(word) 
#wordlis = wordnet_jp.getWordsFromSenses(synonym)

In [6]:
def wordnet_get(word):
    synonym = {}
    words = jp.getWords(word)
    if words:
        for w in words:
            sense = jp.getSenses(w)
            s = jp.getWordsFromSenses(sense)
            synonym = dict(list(synonym.items()) + list(s.items()))

    words_lis = []
    lis = synonym.keys()
    for name in lis:
        for w in synonym[name]:
             words_lis.append(w)
    words_lis = set(words_lis)
    return words_lis

In [7]:
#mecabによる形態素解析(ipadic-neologd),taggerを変えれば何でもok
def tokenize(text):
    cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
    path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,shell=True).communicate()[0]).decode('utf-8')
    m = MeCab.Tagger("-d {0}".format(path))  #mecab-ipadic-neologd
    #m = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
    node = m.parse(text)
    node = node.split("\n")
    noun = []
    for i in range(len(node)):
        word = node[i].split("\t")
        if word[0] == "EOS":
            break
        #品詞に関する分類作業
        word2 = word[1].split(",")
        #print(word2)
        if word2[0] == "名詞" or word2[0] == "形容詞":
            noun.append(word[0])
    
    return noun

In [16]:
#一つのテキストに対する実行例の表示
#処理手順：名詞・形容詞の抽出→抽出した語句がWordnetにあるか→元の文章から該当箇所の変更
text = "この牡蠣とても美味しいからお土産に何個も買った。"
replace_sens = []
noun_lis = tokenize(text)
print(noun_lis)
for n in noun_lis:
    nouns = wordnet_get(n)
    for noun in nouns:
        sample = text.replace(n,noun)
        replace_sens.append(sample)

#文章をリスト化したものに対してすべて処理する関数
def replace_sens_make(sens,num):
    replace_sens = []
    for text in sens:
        candidate = []
        noun_lis = tokenize(text) #形態素解析
        #print(noun_lis)
        for n in noun_lis:
            nouns = wordnet_get(n) #get related words from wordnet
            for noun in nouns:
                sample = text.replace(n,noun) #replace
                candidate.append(sample) #add candidate
        try:
            pickup = random.sample(candidate,num) #新しく生成した文章の中からランダムにn個抽出する
            replace_sens.extend(pickup)
        except:
            print("Error")
            
    return replace_sens

['牡蠣', '美味しい', 'お土産', '何', '個']


In [9]:
#この処理は事前処理：訓練データとテストデータ分割保存用処理
"""
df = pd.read_csv("tag_decition.csv",encoding="utf-8",engine="python")  # データの読み込み
sentences = df.text.values
labels = df.tag.values
X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.2)
df_train = pd.DataFrame({"text": X_train,"tag":y_train})
df_test  = pd.DataFrame({"text": X_test,"tag":y_test})
#df_train.to_csv("train_data.csv")
#df_test.to_csv("test_data.csv")
#df
"""

'\ndf = pd.read_csv("tag_decition.csv",encoding="utf-8",engine="python")  # データの読み込み\nsentences = df.text.values\nlabels = df.tag.values\nX_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.2)\ndf_train = pd.DataFrame({"text": X_train,"tag":y_train})\ndf_test  = pd.DataFrame({"text": X_test,"tag":y_test})\n#df_train.to_csv("train_data.csv")\n#df_test.to_csv("test_data.csv")\n#df\n'

In [30]:
def tag_get(name): #.csvまできちっと書いて！！！
    #load data
    df = pd.read_csv(name,encoding="utf-8",engine="python",usecols=[1,2])
    text_lis = []
    f_text_lis = []
    s_text_lis = []
    for text,tag in zip(df["text"],df["tag"]):
        if tag == 0:
            text_lis.append(text)
        elif tag == 1:
            f_text_lis.append(text)
        else:
            s_text_lis.append(text)
    
    return text_lis,f_text_lis,s_text_lis

In [31]:
safe,flame,spam = tag_get("train_data.csv")

In [32]:
print(len(safe),len(flame),len(spam),sep=" ")

864 600 136


In [40]:
rep_spam = replace_sens_make(spam,7)
rep_spam = random.sample(rep_spam,864)

Error
Error


In [41]:
#ここでrep_spamは生成された文章＋元の文章のリストとなっている
rep_spam += spam
print(len(rep_spam))

1000


In [42]:
#同様に攻撃的文章に対しても実行する
rep_flame = replace_sens_make(flame,1)
print(len(rep_flame))

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
582


In [43]:
rep_flame = random.sample(rep_flame,400)

In [44]:
rep_flame += flame
print(len(rep_flame))

1000


In [45]:
rep_safe = replace_sens_make(safe,1)
rep_safe = random.sample(rep_spam,136)
rep_safe += safe

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [46]:
print(len(rep_safe),len(rep_flame),len(rep_spam),sep=" ")

1000 1000 1000


In [47]:
rep_label = [0]*1000 + [1]*1000 + [2]*1000

In [51]:
all_rep = rep_safe + rep_flame + rep_spam

In [52]:
rep_df = pd.DataFrame({"text": all_rep, "tag": rep_label})

In [54]:
rep_df.to_csv("oversample_text.csv")